# Get Data

In [4]:
import thesillyhome.model_creator.read_config_json as tsh_config
import thesillyhome.model_creator.parse_data as parse_data

import numpy as np
import pandas as pd
import json

In [195]:
import mysql.connector
import psycopg2
from sqlalchemy import create_engine
import uuid

password = 'K3rHUiN6b2bGgeZ'
database = "thesillyhomedb"
user = "admin"
host = "thesillyhomedb-instance-1.cdioawtidgpj.eu-west-2.rds.amazonaws.com"
port = 3306

extdb = create_engine(
            f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}", echo=False
        )

user_id = hex(uuid.getnode())
query = f"SELECT \
                    * \
                from states_snapshots where type = 'chng' limit 3000;"

with extdb.connect() as connection:
    myresult = connection.execute(query).fetchall()


In [12]:
df = pd.read_pickle('3k_data.pickles')

In [18]:
df = df[df['user_id'].isnull()]

In [19]:
df

,id,snapshot_time,get_state_json,type,user_id
0,2,2022-08-18 13:29:49.494421,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
1,13,2022-08-18 13:30:37.813963,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2,14,2022-08-18 13:30:38.157529,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
3,15,2022-08-18 13:30:38.527251,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
4,16,2022-08-18 13:30:38.872546,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
...,...,...,...,...,...
2933,48531,2022-08-20 11:11:29.995995,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2943,48649,2022-08-20 11:12:59.871682,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2944,48651,2022-08-20 11:13:00.115926,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2947,48671,2022-08-20 11:13:14.177513,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None


In [21]:
df_ = df
df_ = df_.drop_duplicates(subset=df_.columns.drop(['id','snapshot_time']))
df_

,id,snapshot_time,get_state_json,type,user_id
0,2,2022-08-18 13:29:49.494421,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
1,13,2022-08-18 13:30:37.813963,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2,14,2022-08-18 13:30:38.157529,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
3,15,2022-08-18 13:30:38.527251,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
4,16,2022-08-18 13:30:38.872546,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
...,...,...,...,...,...
2933,48531,2022-08-20 11:11:29.995995,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2943,48649,2022-08-20 11:12:59.871682,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2944,48651,2022-08-20 11:13:00.115926,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None
2947,48671,2022-08-20 11:13:14.177513,"{""person.lcmchris"": {""entity_id"": ""person.lcmc...",chng,None


In [22]:
df_normalized = pd.json_normalize(df_['get_state_json'].apply(json.loads).to_list())

In [24]:
from thesillyhome.model_creator.parse_data import one_hot_encoder

state_columns = []
for index in df_normalized.columns:
    if 'state' in index:
        if index.rsplit('.',1)[0] in tsh_config.devices:
            state_columns.append(index)

df_normalized = df_normalized[state_columns]
df_normalized = df_normalized.replace(['off',np.NaN, "unknown", "", "unavailable", None],0)
df_normalized = df_normalized.replace(['on'],1)

df_normalized = one_hot_encoder(df_normalized,'device_tracker.chris_phone.state')
df_normalized = one_hot_encoder(df_normalized,'weather.home.state')

act_columns=[]
for index in df_normalized.columns:
    if index.rsplit('.',1)[0] in tsh_config.actuators:
        act_columns.append(index)

sen_columns=[]
for index in df_normalized.columns:
    if index.rsplit('.',1)[0] in tsh_config.sensors:
        if index.rsplit('.',1)[0] not in tsh_config.float_sensors:
            sen_columns.append(index)

 

In [58]:
X = df_normalized[df_normalized.columns.drop(act_columns)]
Y = df_normalized[act_columns]
ratio = int(len(X)*0.8)


(X_train,X_test,Y_train,Y_test) = X[:ratio], X[ratio:],Y[:ratio], Y[ratio:]

In [62]:
X

,binary_sensor.chris_phone_is_charging.state,sensor.bedroom_entrance_sensor_illuminance_lux.state,sensor.livingroom_desk_sensor_illuminance_lux.state,sensor.bathroom_entrance_sensor_illuminance_lux.state,sensor.corridor_end_sensor_illuminance_lux.state,sensor.corridor_entrance_sensor_illuminance_lux.state,binary_sensor.bedroom_entrance_sensor_occupancy.state,binary_sensor.livingroom_desk_sensor_occupancy.state,binary_sensor.bathroom_entrance_sensor_occupancy.state,binary_sensor.corridor_end_sensor_occupancy.state,binary_sensor.corridor_entrance_sensor_occupancy.state,device_tracker.chris_phone.state_home,device_tracker.chris_phone.state_not_home,weather.home.state_cloudy,weather.home.state_partlycloudy,weather.home.state_rainy,weather.home.state_sunny
0,0,626,720,98,18,7,1,1,0,0,0,1,0,0,0,1,0
1,0,626,720,98,18,7,1,1,0,0,0,1,0,0,0,1,0
2,0,626,720,98,18,7,1,1,0,0,0,1,0,0,0,1,0
3,0,626,720,98,18,7,1,1,0,0,0,1,0,0,0,1,0
4,0,626,720,98,18,7,1,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,0,352,642,95,39,17,0,1,0,0,0,1,0,0,1,0,0
1379,0,352,643,95,39,17,0,0,0,0,0,1,0,0,1,0,0
1380,0,352,643,95,39,17,0,0,0,0,0,1,0,0,1,0,0
1381,0,352,632,95,39,17,0,1,0,0,0,1,0,0,1,0,0


In [59]:
import matplotlib.pyplot as plt
from tensorflow import keras

In [60]:
## Model 1
model_dnn = keras.models.Sequential()
model_dnn.add(keras.layers.Dense(50,activation='relu'))
model_dnn.add(keras.layers.Dropout(0.2))
model_dnn.add(keras.layers.Dense(50,activation='relu'))
model_dnn.add(keras.layers.Dropout(0.2))
model_dnn.add(keras.layers.Dense(50,activation='relu'))
model_dnn.add(keras.layers.Dropout(0.2))
model_dnn.add(keras.layers.Dense(len(Y_train.columns),activation='sigmoid'))

nadam = keras.optimizers.Nadam(learning_rate=0.01)
model_dnn.compile(loss='binary_crossentropy',optimizer=nadam,metrics='binary_accuracy')
history = model_dnn.fit(X_train,Y_train, epochs=100,batch_size=5)

pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

model_dnn.evaluate(X_test,Y_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [71]:
## Model 2
n_features = X.shape[1]
n_input = 5
batch_size = 16
model_rnn = keras.models.Sequential()
model_rnn.add(keras.layers.Simp(20,return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model_rnn.add(keras.layers.GRU(20, dropout=0.2, recurrent_dropout=0.2))
model_rnn.add(keras.layers.Dense(len(Y.columns),activation='sigmoid'))

X_new = X.astype(np.float32)
Y_new = Y.astype(np.float32)
Y_new = Y_new.shift(1)
(X_train,X_test,Y_train,Y_test) = X_new[:ratio], X_new[ratio:],Y_new[:ratio], Y_new[ratio:]

generator_train = keras.preprocessing.sequence.TimeseriesGenerator(X_train.values,Y_train.values,
                               length=n_input,
                               batch_size=batch_size)
generator_test = keras.preprocessing.sequence.TimeseriesGenerator(X_test.values,Y_test.values,
                               length=n_input,
                               batch_size=batch_size)


nadam = keras.optimizers.Nadam(learning_rate=0.005)
loss = keras.losses.BinaryCrossentropy()

def last_time_binary_accuracy(Y_true, Y_pred):
  return keras.metrics.binary_accuracy(Y_true[:, -1], Y_pred[:, -1])


model_rnn.compile(loss=loss,optimizer=nadam,metrics=last_time_binary_accuracy)
history = model_rnn.fit(generator_train, epochs=50)

pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

model_rnn.evaluate(generator_test)

Epoch 1/50
69/69 [==============================] - 5s 8ms/step - loss: 0.4632 - last_time_binary_accuracy: 0.6995
Epoch 2/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3713 - last_time_binary_accuracy: 0.7591
Epoch 3/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3376 - last_time_binary_accuracy: 0.7638
Epoch 4/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3292 - last_time_binary_accuracy: 0.7818
Epoch 5/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3249 - last_time_binary_accuracy: 0.7654
Epoch 6/50
69/69 [==============================] - 1s 8ms/step - loss: 0.3206 - last_time_binary_accuracy: 0.7568
Epoch 7/50
69/69 [==============================] - 1s 8ms/step - loss: 0.3150 - last_time_binary_accuracy: 0.7761
Epoch 8/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3194 - last_time_binary_accuracy: 0.7716
Epoch 9/50
69/69 [==============================] - 1s 7ms/step - loss: 0.3066 -

KeyboardInterrupt: 